In [1]:
import pyomo.environ as pe
import pyomo.opt as po

### Modelo Geral

$ \begin{align}
& \max \quad \sum_{i \in \mathbb{N}} c_i x_i\\
& \text{s.t.} \quad \sum_{i \in \mathbb{N}}  a_i x_i \leq b \\
& x_i \in \{0,1\}, \forall i \in \mathbb{N}
\end{align}$

### Modelo exemplo
$ \begin{align}
\max \quad 3x_1 + 4x_2 + 5x_3 + 8x_4 + 9x_5 \\
\text{s.t.} \quad 2x_1 + 3x_2 + 4x_3 + 5x_4 + 9x_5 \leq 20 \\
x_1, x_2, x_3, x_4, x_5 \in \{0,1\}
\end{align}$

In [2]:
solver = po.SolverFactory("glpk")

In [3]:
model = pe.ConcreteModel()

In [4]:
model.N = pe.RangeSet(1,5)

In [5]:
print(set(model.N))

{1, 2, 3, 4, 5}


In [6]:
c = {1:3,2:4,3:5,4:8,5:9} # x1:3 + x2:4 + x3:5 ... função objetivo (max)
a = {1:2,2:3,3:4,4:5,5:9} # x1:2 + x2:3 + x3:4 ... sujeito a restrição
b = 20

In [7]:
model.c = pe.Param(model.N, initialize=c)
model.a = pe.Param(model.N, initialize=a)
model.b = pe.Param(initialize=b)

In [8]:
print(model.c)
print(model.c[2])
print(model.b)
print(model.b.value)

c
4
b
20


In [9]:
model.x = pe.Var(model.N,domain=pe.Binary)

In [10]:
obj_exp = sum(model.c[i] * model.x[i] for i in model.N)
model.obj = pe.Objective(sense=pe.maximize, expr=obj_exp)
print(obj_exp)

3*x[1] + 4*x[2] + 5*x[3] + 8*x[4] + 9*x[5]


In [12]:
con_lhs_expr = sum(model.a[i] * model.x[i] for i in model.N)
con_rhs_expr = model.b
model.con = pe.Constraint(expr=(con_lhs_expr <= con_rhs_expr))

In [13]:
result = solver.solve(model)

In [14]:
for i in model.N:
    print(pe.value(model.x[i]))
print(pe.value(model.obj))

1.0
0.0
1.0
1.0
1.0
25.0
